In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Layer, Input, Embedding, Dense, Dropout, GlobalMaxPooling1D, Concatenate, Masking, LSTM, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LinearSegmentedColormap
import seaborn as sns
from tensorflow.keras.activations import softmax
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
import random
import re
import json
import joblib
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colormaps
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
import matplotlib.cm as cm
import textwrap


class SingleHeadAttention(Layer):
    def __init__(self, embed_dim, proj_dim=None, output_dim=None, return_attention_weights=False, **kwargs):
        super(SingleHeadAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.proj_dim = proj_dim if proj_dim is not None else embed_dim
        self.output_dim = output_dim if output_dim is not None else embed_dim
        self.return_attention_weights = return_attention_weights

        # Projection layers for Q, K, V
        self.q_dense = Dense(self.proj_dim, use_bias=False, name="query_projection")
        self.k_dense = Dense(self.proj_dim, use_bias=False, name="key_projection")
        self.v_dense = Dense(self.proj_dim, use_bias=False, name="value_projection")

        # Output layer
        self.output_dense = Dense(self.output_dim, use_bias=False, name="output_dense")
        self.supports_masking = True

    def call(self, inputs, mask=None):
        q = self.q_dense(inputs)
        k = self.k_dense(inputs)
        v = self.v_dense(inputs)

        # Compute QK^T matrix
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

        # Apply masking logic if provided
        if mask is not None:
            mask = tf.cast(mask, tf.float32)
            mask_expanded = tf.expand_dims(mask, axis=-1)
            matmul_mask = tf.matmul(mask_expanded, mask_expanded, transpose_b=True)
            scaled_attention_logits += (1 - matmul_mask) * -1e9

        # Compute attention weights
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        #attention_weights *= matmul_mask

        # Compute the attention output
        attention_output = tf.matmul(attention_weights, v)
        output = self.output_dense(attention_output)

        if self.return_attention_weights:
            return output, attention_weights
        return output

    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'proj_dim': self.proj_dim,
            'output_dim': self.output_dim,
            'return_attention_weights': self.return_attention_weights,
            'supports_masking': self.supports_masking
        })
        return config

# Create the model
def create_model(max_lengths, vocab_size, embedding_dim, proj_dim, output_dim, include_attention_weights=False):
    inputs = []
    main_outputs = []
    attention_weights_outputs = []

    for i, max_len in enumerate(max_lengths):
        input_layer = Input(shape=(max_len,), dtype='int32', name=f"text_input_{i+1}")
        embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True, trainable=True)(input_layer)

        # Bidirectional LSTM layer
        lstm_output = Bidirectional(LSTM(16, return_sequences=True))(embedding_layer)

        # Single-head self-attention layer
        attention_layer = SingleHeadAttention(embed_dim=embedding_dim, proj_dim=proj_dim, output_dim=output_dim, 
                                              return_attention_weights=include_attention_weights)

        if include_attention_weights:
            attention_output, attention_weights = attention_layer(lstm_output)
            attention_weights_outputs.append(attention_weights)
        else:
            attention_output = attention_layer(lstm_output)

        # Apply Global Max Pooling to reduce the sequence dimension
        pooled_output = GlobalMaxPooling1D()(attention_output)
        main_outputs.append(pooled_output)
        inputs.append(input_layer)

    # Numeric input
    numeric_input = Input(shape=(18,), dtype='float32', name="numeric_input")
    inputs.append(numeric_input)
    combined_features = Concatenate()([*main_outputs, numeric_input])

    # Fully connected layers
    x = Dense(128, activation='relu', kernel_initializer='he_normal')(combined_features)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu', kernel_initializer='he_normal')(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu', kernel_initializer='he_normal')(x)
    x = Dropout(0.2)(x)
    final_output = Dense(2)(x)

    # Compile the model
    model = Model(inputs=inputs, outputs=[final_output] + attention_weights_outputs if include_attention_weights else [final_output])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')

    return model

# Define file paths
output_dir = 'E:/Concrete_Freeze_Thaw_Damage_Prediction_Results/LSTM-SA_outputs'
vocab_file = os.path.join(output_dir, 'vocab_index.json')
max_lengths_file = os.path.join(output_dir, 'max_lengths.csv')
scaler_file = os.path.join(output_dir, 'scaler.pkl')
model_weights_file = os.path.join(output_dir, 'model_fold_4.h5')
vocab_size = 2025

# Load vocabulary index
with open(vocab_file, 'r', encoding='utf-8') as f:
    word_index = json.load(f)

# Load maximum sequence lengths
max_lengths_df = pd.read_csv(max_lengths_file)
max_lengths = max_lengths_df['Max_Length'].tolist()

# Load scaler for numeric features
scaler = joblib.load(scaler_file)

# Create model, ensuring the parameters are the same as training. Set include_attention_weights=True to obtain attention weights.
analysis_model = create_model(max_lengths, vocab_size, embedding_dim=64,  proj_dim=32, output_dim=32, include_attention_weights=True)

# Load model weights
analysis_model.load_weights(model_weights_file)

# Define new sample texts and numeric data
new_sample_text = [
    'water', 
    'The freeze-thaw cycle test method uses the rapid freeze-thaw method. The specimens are maintained in the standard maintenance room until 24 d and then removed and immersed in water for 4 d to saturate them with water. During the freeze-thaw test, the water surface in the specimen box is higher than the upper surface of the specimen by 5 mm. The maximum temperature in the center of the specimen is controlled at (5 ± 2) ℃, and the minimum temperature is controlled at (? 18 ± 2) ℃. At any time, the temperature in the center of the specimen should not be higher than 7 ℃ and not lower than ? 20 ℃. The maximum number of freeze-thaw cycles is designed to be 100, the specimens are weighed, and their dynamic modulus of elasticity is measured after every 25 freeze-thaw cycles. The distance from the loading point to the support point is 150 mm.',
    'The microstructure of the interface transition zone under waste brick concrete, characterized by SEM, is shown in Fig. 5. Before the freeze-thaw cycle, the interface transition zone region of the waste brick concrete has many hydrates, as seen in Fig. 5(a), of which the main component is calcium silicate hydrate (C-S-H). The moderately dense connections between the hydrogels contribute to the bond strength between the coarse aggregates and the cement matrix. As a result, cracks appear in the interface transition zone of WCB, the internal C-S-H content decreases, the cohesion between the cement mortar and the coarse aggregate is reduced, and the interface transition zone is significantly deteriorated after 100 freeze-thaw cycles, as shown in Fig. 5(b). In addition, a large amount of ettringite is present in the interface transition zone of NCA after 100 freeze-thaw cycles, as shown in Fig. 5(c), which led to a decrease in the interfacial cohesion of the cement paste.',
    'The influence of waste brick coarse aggregate on the overall concrete can be described in two aspects. On one hand, the interior of waste brick coarse aggregate is loose and porous, and the interconnected capillary pores allow water to enter the aggregate and cause freeze-thaw damage, which is unsuitable for its frost resistance. On the other hand, there are some closed pores in the waste brick coarse aggregate. The pores can play a role similar to an air-entraining agent and improve the frost resistance of concrete. However, the pre-wetting of the aggregate in this experiment will significantly reduce the number of closed pores, so the improvement effect on the frost resistance of concrete is limited.'
]  

new_sample_numeric = [0.4,0,487,0,0,0,195,613,0,900,15,16.3,0,0,0,0,100,1] 

# Text preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[,.:]', '', text)
    return text

# Tokenize, sequence, and pad new sample texts
new_sample_sequences = []

for i, text in enumerate(new_sample_text):
    processed_text = preprocess_text(text)
    tokens = word_tokenize(processed_text)
    sequence = [word_index.get(word, 0) for word in tokens]
    padded_sequence = pad_sequences([sequence], maxlen=max_lengths[i], padding='post', truncating='post')[0]
    new_sample_sequences.append(padded_sequence)

# Scale numeric data
new_sample_numeric_scaled = scaler.transform(np.array([new_sample_numeric], dtype=np.float32))

# Combine inputs (text sequences and numeric features)
new_sample_input = [np.array([seq], dtype=np.float32) for seq in new_sample_sequences] + [new_sample_numeric_scaled]

# Make model prediction
outputs = analysis_model.predict(new_sample_input)

# Output prediction results and attention weights

# Extract output for analysis
final_output = outputs[0]
sequence_outputs = outputs[1:]  # Sequence outputs for each text input
print(final_output)
# Print shapes of each sequence output
for i, seq_output in enumerate(sequence_outputs):
    print(f"Sequence output {i + 1} shape:", seq_output.shape)



# Data processing function
def process_data(df):
    # 1. Remove rows where all numeric values are exactly equal
    df = df.loc[~(df.eq(df.iloc[:, 0], axis=0)).all(axis=1)]
    
    # 2. Remove columns which are entirely 0
    df = df.loc[:, (df != 0).any(axis=0)]
    
    # 3. Warn if the number of rows and columns are not equal
    if df.shape[0] != df.shape[1]:
        print(f"Warning: After processing, the number of rows ({df.shape[0]}) does not equal the number of columns ({df.shape[1]})")
    
    return df

# Save path
save_path = r"E:/Concrete_Freeze_Thaw_Damage_Prediction_Results/LSTM-SA_outputs/Visualization/outputs.xlsx"

# Save final_output to a DataFrame
final_output_df = pd.DataFrame(final_output)

# Create an Excel writer object and save each output
with pd.ExcelWriter(save_path, engine='openpyxl') as writer:
    # Process and save final_output
    final_output_processed = process_data(final_output_df)
    final_output_processed.to_excel(writer, sheet_name='Final Output', index=False)
    
    # Process and save each sequence output
    for i, seq_output in enumerate(sequence_outputs):
        seq_output_df = pd.DataFrame(seq_output.reshape(-1, seq_output.shape[-1]))  # Reshape to 2D
        processed_seq_output = process_data(seq_output_df)  # Process data
        processed_seq_output.to_excel(writer, sheet_name=f'Sequence Output {i + 1}', index=False)

print(f"Processed outputs saved to {save_path}")


# Load the Excel file
excel_path = 'E:/Concrete_Freeze_Thaw_Damage_Prediction_Results/LSTM-SA_outputs/Visualization/outputs.xlsx'
data_from_sheets = pd.read_excel(excel_path, sheet_name=None)

# Define the output directory
output_dir = r"E:/Concrete_Freeze_Thaw_Damage_Prediction_Results/LSTM-SA_outputs/Visualization"

# Modified processing and visualization function
def process_and_visualize(sheet_name, text, output_dir, csv_filename, svg_filename):
    # Retrieve the data from the specified sheet
    current_sheet_data = data_from_sheets[sheet_name]
    attention_scores_matrix = current_sheet_data.values  # Use all data as the attention matrix

    # Get tokens from the text
    tokens = word_tokenize(preprocess_text(text))
    print(f"Tokens count in {sheet_name}: {len(tokens)}")

    # Calculate the average attention scores for each column
    row_mean_scores = np.mean(attention_scores_matrix, axis=0)

    # Normalize the scores
    normalized_scores = (row_mean_scores - np.min(row_mean_scores)) / (np.max(row_mean_scores) - np.min(row_mean_scores))

    # Save normalized scores to CSV
    save_path_csv = os.path.join(output_dir, csv_filename)
    normalized_scores_df = pd.DataFrame({
        'Token': tokens,  # Assuming the number of tokens and normalized_scores are the same
        'Normalized Score': normalized_scores
    })
    normalized_scores_df.to_csv(save_path_csv, index=False, encoding='utf-8-sig')
    print(f"Saved CSV: {save_path_csv}")

    # Use a colormap for the text
    cmap = plt.colormaps['Blues']
    color_mapped_scores = [cmap(score) for score in normalized_scores]  # Get color for each score

    # Create the plotting area, ensuring sufficient width to avoid word-wrapping
    fig, ax = plt.subplots(figsize=(10, 4))

    # Set a maximum width for each line (adjust as needed)
    wrapper = textwrap.TextWrapper(width=95)
    wrapped_text = wrapper.wrap(" ".join(tokens))  # Wrap text into multiple lines

    # Set up initial text coordinates and line height
    y_pos = 0
    line_height = 0.03

    # Draw each line of wrapped text with appropriate word colors
    for line in wrapped_text:
        words = line.split()
        x_pos = 3  # Starting x-coordinate for each line
        for word in words:
            if word in tokens:
                original_idx = tokens.index(word)  # Get the index in the original tokens list
                score = normalized_scores[original_idx]  # Get the corresponding attention score
                ax.text(x_pos, y_pos, word,
                        fontsize=12, fontweight='bold', family='Times New Roman',
                        color=cmap(score), ha='left', va='center')
            x_pos += len(word) + 1  # Update x-coordinate (1 represents a space)
        y_pos -= line_height  # Move down one line

    # Set plot limits and style
    ax.axis([0, 100, y_pos, 0])
    ax.axis('off')  # Turn off the axes

    # Save the figure as SVG
    save_path_svg = os.path.join(output_dir, svg_filename)
    plt.savefig(save_path_svg, format='svg', dpi=300)
    print(f"Saved SVG: {save_path_svg}")

    # Display the plot
    plt.show()

    # Return tokens count and number of score columns for additional info
    return len(tokens), attention_scores_matrix.shape[1]

# Define the sheets and corresponding text data and file names
tasks = [
    {"sheet_name": "Sequence Output 1", "text": new_sample_text[0], "csv": "normalized_scores_sequence_output_1.csv", "svg": "Column_1.svg"},
    {"sheet_name": "Sequence Output 2", "text": new_sample_text[1], "csv": "normalized_scores_sequence_output_2.csv", "svg": "Column_2.svg"},
    {"sheet_name": "Sequence Output 3", "text": new_sample_text[2], "csv": "normalized_scores_sequence_output_3.csv", "svg": "Column_3.svg"},
    {"sheet_name": "Sequence Output 4", "text": new_sample_text[3], "csv": "normalized_scores_sequence_output_4.csv", "svg": "Column_4.svg"},
]

# Process each task
for task in tasks:
    tokens_count, scores_columns = process_and_visualize(
        sheet_name=task["sheet_name"],
        text=task["text"],
        output_dir=output_dir,
        csv_filename=task["csv"],
        svg_filename=task["svg"]
    )
    print(f"Attention scores columns for {task['sheet_name']}: {scores_columns}")
    print(f"Tokens count for {task['sheet_name']}: {tokens_count}")
